In [121]:
import json
import pandas as pd
import numpy as np
from IPython.display import IFrame, display
    
def float_formatter(x): return '{:.3f}'.format(x)

def highlight_max(s):
    if (s.dtype == object):
        is_max = [False for _ in range(s.shape[0])]
    else:
        is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]

wanted_keys = ("internal", "external")
dictfilt = lambda x, y: dict([ (i,'{:.3f}'.format(float(x[i]))) for i in x if i in set(y) ])

def retrieve_single_result_cnv():
    f = open(f'Result/Many_Feature_Selection_notebook/cnv/cnv.json')
    data = json.load(f)
    new_dict = {}
    new_dict['cnv'] = {}
    new_dict['cnv']['spearman'] = dictfilt(data['spearman'], wanted_keys)
    return pd.DataFrame(new_dict)

def print_best_estimator(dataset, feature_selection):
    f = open(f'Result/Many_Feature_Selection_notebook/{dataset}/{dataset}.json')
    data = json.load(f)
    print(data[feature_selection]['best_estimator'])
    
def retrieve_results():
    new_dict = {}
    for dataset in ['mirna', 'proteins', 'mrna', 'whole_dataset']:
        new_dict[dataset] = {}
        f = open(f'Result/Many_Feature_Selection_notebook/{dataset}/{dataset}.json')
        data = json.load(f)
        for metodo in ['mannwhitneyu','mannwhitneyu_with_umap', 'boruta', 'mine', 'mrmr']:
            if metodo in data:
                new_dict[dataset][metodo] = dictfilt(data[metodo], wanted_keys)
            else: 
                new_dict[dataset][metodo] = -1
    return pd.DataFrame(new_dict)

def load_filtered_normalize_data():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins']
    dict_test = {}
    for data in dataset_dict:
        with open('filtered_normalize_data/'+data+'.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

def load_data_after_r_function():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins',
                    'dataset_whole_dataset', 'proteins_mirna_dataset',
                   'proteins_mirna_mrna_dataset']
    dict_test = {}
    for data in dataset_dict:
        with open('elaborated_data_with_r_function/r_'+data+'_removed.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

lista_dataset = ['proteins', 'cnv', 'mrna', 'mirna','whole_dataset',
                'proteins_mirna','proteins_mirna_mrna']

lista_path_on_whole = ['Experiments_using_R_Function_intrinsic_and_umap_whole_dataset',
             'Experiments_using_R_Function_intrinsic_and_tsne_whole_dataset',
             'Experiments_using_R_Function_mannwhitney_and_multivariate_feature_selection_whole_dataset']

lista_path_on_fold = ['Experiments_using_R_Function_whole_intrinsic_and_umap_on_fold',
             'Experiments_using_R_Function_whole_intrinsic_and_tsne_on_fold',
             'Experiments_using_R_Function_mannwhitney_and_multivariate_feature_selection_on_fold']

def retrieve_results_on_fold_or_on_whole(lista_path, lista_dataset):
    new_dict = {}
    for dataset in lista_dataset:
        new_dict[dataset] = {}
        for path in lista_path:
            if 'umap' in path:
                f = open(f'Result/{path}/r_function_intrinsic_with_umap/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_intrinsic_with_umap'] = dictfilt(
                    data['r_function_intrinsic_with_umap'], wanted_keys)
            elif 'tsne' in path:
                f = open(f'Result/{path}/r_function_intrinsic_with_tsne/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_intrinsic_with_tsne'] = dictfilt(
                    data['r_function_intrinsic_with_tsne'], wanted_keys)
            elif 'multivariate' in path:
                if dataset != 'cnv':
                    
                    f = open(f'Result/{path}/r_function_with_mannwhitney_boruta/{dataset}.json')
                    data = json.load(f)
                    new_dict[dataset]['r_function_with_mannwhitney_boruta'] = dictfilt(
                        data['r_function_with_mannwhitney_boruta'], wanted_keys)
                    
                if dataset != 'mrna' and dataset != 'whole_dataset' and dataset != 'proteins_mirna_mrna':
                    
                    f = open(f'Result/{path}/r_function_with_mannwhitney_mrmr/{dataset}.json')
                    data = json.load(f)
                    new_dict[dataset]['r_function_with_mannwhitney_mrmr'] = dictfilt(
                        data['r_function_with_mannwhitney_mrmr'], wanted_keys)
                
                f = open(f'Result/{path}/r_function_with_mannwhitney/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_with_mannwhitney'] = dictfilt(
                    data['r_function_with_mannwhitney'], wanted_keys)
    return pd.DataFrame(new_dict)

datasets = load_filtered_normalize_data()

data_after_r_function = load_data_after_r_function()

#create combination of dataset
dataset_whole_dataset = pd.concat([datasets['cnv'], datasets['proteins'], datasets['mrna'], datasets['mirna']], axis=1)
datasets['dataset_whole_dataset'] = dataset_whole_dataset

proteins_mirna_dataset = pd.concat([datasets['proteins'], datasets['mirna']], axis=1)
datasets['proteins_mirna_dataset'] = proteins_mirna_dataset

proteins_mirna_mrna_dataset = pd.concat([datasets['proteins'], datasets['mirna'], datasets['mrna']], axis=1)
datasets['proteins_mirna_mrna_dataset'] = proteins_mirna_mrna_dataset

# Osservazioni iniziali

1) Inizialmente tutti i i dati sono stati normalizzati usando MinMaxScaler e filtrati scartando le feature con bassa variabilità ( deviazione standard < 0.05)
3) Il modello usato è sempre RandomForest
3) Viene usata l'AUPRC come misuratore di performance. La baseline per il dataset fornito è 0.122807, ovvero il 12% dei pazienti sono positivi (77 su 627).
4) Viene sempre applicata una StratifiedKFold usando 10 fold.
5) I risultati sono sempre presentati con etichetta "external" e "internal". Questo sta a indicare quando è stata calcolata la AUPRC: nel primo caso le predizioni effettuate dal modello nei 10 fold vengono concatenate e una volta finite viene calcolata l'AUPRC totale; nel secondo caso invece ad ogni iterazione viene calcolata l'AUPRC e alla fine viene calcolata la media. Calcolando l'AUPRC esternamente sono anche in grado di avere un grafico, cosa che internamente non è possibile poiché avrei 10 grafici distinti.

### A cnv ho applicato spearman e scartato le feature altamente correlate tra di loro o bassamente correlate con la label

In [107]:
retrieve_single_result_cnv()

,cnv
spearman,"{'internal': '0.135', 'external': '0.099'}"


In [108]:
print_best_estimator('cnv', 'spearman')

RandomForestClassifier(criterion='entropy', max_leaf_nodes=5, n_estimators=101)


### Figura

In [109]:
filepath = "Result/Many_Feature_Selection_notebook/cnv/spearman_AUPRC_External.pdf"
IFrame(filepath, width=700, height=400)

# Risultati di proteins, miRNA e mRNA e concat (concatenazione di tutti i dataset)

In [110]:
retrieve_results()

,mirna,proteins,mrna,whole_dataset
mannwhitneyu,"{'internal': '0.249', 'external': '0.165'}","{'internal': '0.223', 'external': '0.147'}","{'internal': '0.228', 'external': '0.148'}","{'internal': '0.228', 'external': '0.156'}"
mannwhitneyu_with_umap,"{'internal': '0.269', 'external': '0.118'}","{'internal': '0.263', 'external': '0.115'}","{'internal': '0.387', 'external': '0.143'}","{'internal': '0.214', 'external': '0.113'}"
boruta,"{'internal': '0.181', 'external': '0.136'}","{'internal': '0.167', 'external': '0.119'}","{'internal': '0.199', 'external': '0.161'}","{'internal': '0.220', 'external': '0.155'}"
mine,"{'internal': '0.249', 'external': '0.175'}","{'internal': '0.212', 'external': '0.155'}","{'internal': '0.227', 'external': '0.166'}",-1
mrmr,"{'internal': '0.198', 'external': '0.138'}","{'internal': '0.191', 'external': '0.144'}",-1,-1


#### Per mRNA e concat in corrispodenza di mrmr c'è -1, questo perchè il calcolo richiedeva tantissimo tempo, sopratutto usando 10 fold

## Best estimator of the best combination of feature selection + extraction

In [111]:
print_best_estimator('proteins', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_features='log2',
                       max_leaf_nodes=5, n_estimators=51)


In [112]:
print_best_estimator('mrna', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_leaf_nodes=10, n_estimators=51)


In [113]:
print_best_estimator('mirna', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_features='log2',
                       max_leaf_nodes=10, n_estimators=51)


In [114]:
print_best_estimator('whole_dataset', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_leaf_nodes=10, n_estimators=51)


# Seconda tipologia di esperimenti

### Come prima cosa viene utlizzata una funzione che scarta le feature altamente correlate tra di loro usando "pearson" con un cutoff = 0.8. Vediamo quante feature vengono scartate:

In [115]:
for dataset in ['proteins', 'cnv', 'mirna', 'mrna',
                'dataset_whole_dataset', 'proteins_mirna_dataset',
               'proteins_mirna_mrna_dataset']:    
    print(f"{dataset}: feature prima dell applicazione della funzione: {len(datasets[dataset].columns)}")
    print(f"{dataset}: feature dopo l applicazione della funzione: {len(data_after_r_function[dataset].columns)}")
    print(f"{dataset}: feature eliminate: {len(datasets[dataset].columns)-len(data_after_r_function[dataset].columns)}")
    print('\n')

proteins: feature prima dell applicazione della funzione: 216
proteins: feature dopo l applicazione della funzione: 216
proteins: feature eliminate: 0


cnv: feature prima dell applicazione della funzione: 24776
cnv: feature dopo l applicazione della funzione: 194
cnv: feature eliminate: 24582


mirna: feature prima dell applicazione della funzione: 773
mirna: feature dopo l applicazione della funzione: 761
mirna: feature eliminate: 12


mrna: feature prima dell applicazione della funzione: 18465
mrna: feature dopo l applicazione della funzione: 18411
mrna: feature eliminate: 54


dataset_whole_dataset: feature prima dell applicazione della funzione: 44230
dataset_whole_dataset: feature dopo l applicazione della funzione: 20386
dataset_whole_dataset: feature eliminate: 23844


proteins_mirna_dataset: feature prima dell applicazione della funzione: 989
proteins_mirna_dataset: feature dopo l applicazione della funzione: 977
proteins_mirna_dataset: feature eliminate: 12


proteins_mirna_m

### La differenza tra on fold e on whole dataset:
1) In entrambi i casi la funzione che calcola la correlazione tra le feature viene applicata sull'intero dataset, prima di darlo in pasto alla funzione che splitta i dati in fold e addestra
2) In 'on fold' il calcolo della dimensione intrinseca viene effettuato sul train set del fold generato durante la StratifiedKFold e la dimensione calcolata viene usata per fare feature extraction sia sul train set che sul test set, usando tSNE e Umap.
3) In 'on whole dataset' invece la dimensione intrinseca viene calcolata esternamente alla funzione, quindi su tutto il dataset, e anche la feature extraction viene effettuata esternamente. Il dataset cosi ottenuto viene passato alla funzione che addestra.

# On fold

In [122]:
retrieve_results_on_fold_or_on_whole(lista_path_on_fold, lista_dataset).style.apply(highlight_max, axis=None).format(float_formatter)

KeyError: 'internal'

### whole dataset (transductive)

In [119]:
retrieve_results_on_fold_or_on_whole(lista_path_on_whole, lista_dataset)

,proteins,cnv,mrna,mirna,whole_dataset,proteins_mirna,proteins_mirna_mrna
r_function_intrinsic_with_umap,"{'internal': '0.719', 'external': '0.694'}","{'internal': '0.552', 'external': '0.563'}","{'internal': '0.633', 'external': '0.637'}","{'internal': '0.787', 'external': '0.785'}","{'internal': '0.703', 'external': '0.684'}","{'internal': '0.735', 'external': '0.733'}","{'internal': '0.699', 'external': '0.679'}"
r_function_intrinsic_with_tsne,"{'internal': '0.136', 'external': '0.108'}","{'internal': '0.153', 'external': '0.115'}","{'internal': '0.168', 'external': '0.114'}","{'internal': '0.206', 'external': '0.153'}","{'internal': '0.164', 'external': '0.140'}","{'internal': '0.194', 'external': '0.139'}","{'internal': '0.179', 'external': '0.141'}"
r_function_with_mannwhitney_boruta,"{'internal': '0.245', 'external': '0.175'}",NaN,"{'internal': '0.383', 'external': '0.292'}","{'internal': '0.287', 'external': '0.179'}","{'internal': '0.388', 'external': '0.303'}","{'internal': '0.302', 'external': '0.239'}","{'internal': '0.380', 'external': '0.324'}"
r_function_with_mannwhitney_mrmr,"{'internal': '0.235', 'external': '0.167'}","{'internal': '0.255', 'external': '0.164'}",NaN,"{'internal': '0.253', 'external': '0.180'}",NaN,"{'internal': '0.287', 'external': '0.215'}",NaN
r_function_with_mannwhitney,"{'internal': '0.250', 'external': '0.188'}","{'internal': '0.244', 'external': '0.165'}","{'internal': '0.264', 'external': '0.178'}","{'internal': '0.223', 'external': '0.177'}","{'internal': '0.265', 'external': '0.158'}","{'internal': '0.312', 'external': '0.208'}","{'internal': '0.248', 'external': '0.173'}"
